In [ ]:
import mne
import matplotlib.pyplot as plt
import tensorflow as tf

import data_preprocessing as dp
import training
import utilities

In [ ]:
Adquisition_eq="biosemi128"
montage = mne.channels.make_standard_montage(Adquisition_eq)
montage.plot()

# https://www.frontiersin.org/articles/10.3389/fneur.2020.00375/full

## Dataset
Data is epoched, keeping only the 4.5s length signals correpsonding to the time window between the beginning of the concentration interval and the end of the relacation interval.


In [ ]:
data, events = dp.load_data()

In [ ]:
import numpy as np
data, events = dp.choose_condition(data, events, 'inner speech')
data = data.astype(np.float16) * 100_000  # NORMALIZATION ONLY POSSIBLE BY MULTIPLYING HERE?

In [ ]:
data = dp.normalization(data)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data, events[:, 1]))
dataset = dp.preprocessing_pipeline(dataset, dp.filter_interval, [[1, 3.5], 256], batch_size=12)

In [ ]:
datasets = dp.split_dataset(dataset)

### Display Data

In [ ]:
for data_batch, target_batch in datasets['test'].take(1):
    utilities.plot_eeg_data(data_batch[0])

### Model-Testing

In [ ]:
class ConvNet(tf.keras.Model):
    def __init__(self, layer_list=[
        tf.keras.layers.Conv2D(filters=64, kernel_size=3),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=64, kernel_size=5),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.ReLU(),
        tf.keras.layers.Conv2D(filters=128, kernel_size=6),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(50, 'sigmoid'),
        tf.keras.layers.Dense(10, 'sigmoid'),
        tf.keras.layers.Dense(4, 'softmax'),
    ]) -> None:
        super(ConvNet, self).__init__()
        self.layer_list = layer_list
    
    @tf.function
    def call(self, inputs) -> tf.Tensor:
        x = inputs
        for layer in self.layer_list:
            x = layer(x)
        return x

In [ ]:
for key, value in datasets.items():
    datasets[key] = value.map(lambda i, t: (tf.reshape(i, (12, 128, 640, 1)), t))


In [ ]:
# model
net1 = ConvNet()
# Initialize the loss-function
cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()
# Initialize the optimizer
optimizer = tf.keras.optimizers.Adam()

losses_res, accuracies_res = training.training(net1, datasets, cross_entropy_loss,
                                      optimizer, epochs=30)